# MongoDB intro

In [1]:
# Read user and password
with open('.streamlit\secrets.toml', 'r') as file:
    text = file.read()

import streamlit as st
import pymongo


def init_connection():
    return pymongo.MongoClient(st.secrets["mongo"]["uri"])

client = init_connection()


In [2]:
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [3]:
# Selecting a database and a collection.
database = client['example']
collection = database['data']

In [4]:
# Inserting a single document (dictionary).
collection.insert_one({'name': 'Hallvard', 'age': 23})

# Inserting multiple documents (list of dictionaries).
collection.insert_many([
    {'name': 'Kristian', 'age': 27},
    {'name': 'Ihn Duck', 'age': 15},
#    {'name': 'Ihn Duck', 'age': 16},
])

# Note that an _id field is automatically generated by MongoDB.

In [5]:
# Reading ALL documents from a collection.
# ........................................

documents = collection.find({})
# A cursor is returned.

# The cursor can be iterated over:
for document in documents:
    print(document)

# Or directly converted to a list:
#documents = list(documents)

{'_id': ObjectId('68fbb30030cb1b23e0b5d859'), 'name': 'Hallvard', 'age': 23}
{'_id': ObjectId('68fbb30030cb1b23e0b5d85a'), 'name': 'Kristian', 'age': 27}
{'_id': ObjectId('68fbb30030cb1b23e0b5d85b'), 'name': 'Ihn Duck', 'age': 15}


In [6]:
# Reading SPECIFIC documents from a collection.
# .............................................

hallvard = collection.find({'name': 'Hallvard'})

for document in hallvard:
    print(document)

hallvard = list(hallvard)

{'_id': ObjectId('68fbb30030cb1b23e0b5d859'), 'name': 'Hallvard', 'age': 23}


In [7]:
# Updating a single document.
# ...........................
collection.update_one(
    {'name': 'Hallvard'},
    {'$set': {'name': 'Hallvard Lavik'}}  # Sets the `name` to `Hallvard Lavik`.
)

# Updating multiple documents.
# ............................
collection.update_many(
    {},
       {'$inc': {'age': 1}}  # Increments the `age` of all documents by `1`.
)

In [8]:
pipeline = [
    {'$match': {'age': {'$gt': 20}}},
    {'$group': {'_id': None, 'average_age_over_20': {'$avg': '$age'}}},
]
result = collection.aggregate(pipeline)
result = list(result)
print(result)

[{'_id': None, 'average_age_over_20': 26.0}]


In [9]:
# Deleting a single document.
# ...........................
collection.delete_one({'name': 'Ihn Duck'})  # Deletes the document with `name = Ihn Duck`.

In [10]:
# Deleting multiple documents.
# ............................
collection.delete_many({'age': {'$gt': 25}})  # Deletes documents where `age > 25`.

# Deleting all documents.
# .......................
collection.delete_many({})